In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
import joblib

In [2]:
# 1. Load data
df = pd.read_csv('../Servo_Mechanism.csv', on_bad_lines='skip')
X = df.drop('Class', axis=1)
y = df['Class']

In [3]:
# 2. Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
# 3. Preprocessing
preprocessor = ColumnTransformer([
    ('cat', OneHotEncoder(handle_unknown='ignore'), ['Motor', 'Screw']),
    ('num', StandardScaler(), ['Pgain', 'Vgain'])
])

In [5]:
# 4. Pipeline + hyperparameter grid
pipeline = Pipeline([
    ('pre', preprocessor),
    ('model', SVR())
])

param_grid = {
    'model__kernel': ['rbf'],
    'model__C': [0.1, 1, 10, 50, 100],
    'model__gamma': ['scale', 0.01, 0.1, 1],
    'model__epsilon': [0.1, 0.2, 0.5]
}

grid_svr = GridSearchCV(
    pipeline,
    param_grid,
    cv=5,
    scoring='r2',
    n_jobs=-1,
    verbose=1
)

In [6]:
# 5. Train & evaluate
grid_svr.fit(X_train, y_train)
print("Best SVR params:", grid_svr.best_params_)

Fitting 5 folds for each of 60 candidates, totalling 300 fits
Best SVR params: {'model__C': 50, 'model__epsilon': 0.5, 'model__gamma': 'scale', 'model__kernel': 'rbf'}


In [8]:
y_pred_svr = grid_svr.predict(X_test)
print("SVR:")
print(f"  MSE: {mean_squared_error(y_test, y_pred_svr):.3f}")
print(f"  R2 : {r2_score(y_test, y_pred_svr):.3f}")

SVR:
  MSE: 40.674
  R2 : 0.827


In [9]:
joblib.dump(pipeline, 'svr_model.pkl')

['svr_model.pkl']